In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Read stackoverflow survey dataset
df = pd.read_csv("./data/survey_results_public.csv")
df.head()

In [ ]:
df.columns

In [ ]:
df = df[["Country", "EdLevel", "YearsCode", "Employment", "ConvertedCompYearly"]]
df = df.rename({"ConvertedCompYearly": "Salary"}, axis=1)
df.head()

In [ ]:
df = df[df["Salary"].notnull()]
df.info()

In [ ]:
df.isnull().sum()

In [ ]:
df_no_na = df.dropna()
df_no_na.isnull().sum()

In [ ]:
df_no_na["Employment"]

In [ ]:
df_fulltime_employment = df_no_na[df_no_na["Employment"]
                                  == "Employed, full-time"]
df_drop_employment = df_fulltime_employment.drop("Employment", axis=1)
df_drop_employment.info()

In [ ]:
df_drop_employment.count()

In [ ]:
df_drop_employment["Country"].value_counts()

In [ ]:
def shorted_categories(categories, cutoff_value):
    categorical = {}
    for i in range(len(categories)):
        if categories.values[i] >= cutoff_value:
            categorical[categories.index[i]] = categories.index[i]
        else:
            categorical[categories.index[i]] = "Other"
    return categorical

In [ ]:
country_map = shorted_categories(
    df_drop_employment.Country.value_counts(), 400)
df_drop_employment["Country"] = df_drop_employment["Country"].map(country_map)
df_drop_employment.Country.value_counts()

In [ ]:
df_drop_employment["Country"].isnull().sum()

In [ ]:
# Explore statistic of cloumn salary
print(df_drop_employment["Salary"].min())
print(df_drop_employment["Salary"].max())
print(df_drop_employment["Salary"].mean())

In [ ]:
# Clean extreme value

# 3 sigma rule
salary_mean = df_drop_employment["Salary"].mean()
salary_std = df_drop_employment["Salary"].std()
print(f"Salary mean : {salary_mean}")
print(f"Salary std : {salary_std}")
tsig_upper_bound = salary_mean + (3 * salary_std)
tsig_lower_bound = salary_mean - (3 * salary_std)
print(f"3 sigma upper bound : {tsig_upper_bound}")
print(f"3 sigma lower bound : {tsig_lower_bound}")

# IQR
q1 = df_drop_employment["Salary"].quantile(0.25)
q3 = df_drop_employment["Salary"].quantile(0.75)
iqr = q3 - q1
print(f"IQR : {iqr}")
iqr_upper_bound = q3 + 1.5 * iqr
iqr_lower_bound = q1 - 1.5 * iqr
print(f"IQR upper bound {iqr_upper_bound}")
print(f"IQR lower bound {iqr_lower_bound}")

# Winsorizing
upper_limit = df_drop_employment["Salary"].quantile(0.99)
lower_limit = df_drop_employment["Salary"].quantile(0.01)
print(f"Winsorizing upper limit : {upper_limit}")
print(f"Winsorizing lower limit : {lower_limit}")

# Choose upper limit and lower limit from winsorizing to filter out extreme value
df_clean_salary = df_drop_employment[df_drop_employment["Salary"].between(
    lower_limit, upper_limit)]
print(df_clean_salary["Salary"].min())
print(df_clean_salary["Salary"].max())
print(df_clean_salary["Salary"].mean())

In [ ]:
df_clean_salary.count()

In [ ]:
# Inspect sarary range with Boxplot
fig, ax = plt.subplots(1, 1, figsize=(12, 7))
df_clean_salary.boxplot("Salary", "Country", ax=ax)
plt.xticks(rotation=90)
plt.xlabel("Country")
plt.ylabel("Salary")
plt.show()

In [ ]:
df_clean_salary["Country"].unique()

In [ ]:
df_clean_salary[df_clean_salary["Country"] == "Other"].count()

In [ ]:
df_clean_other = df_clean_salary[df_clean_salary["Country"] != "Other"]
df_clean_other.count()

In [ ]:
df_clean_other["Country"].unique()

In [ ]:
# Clean year experience
df_clean_other["YearsCode"].unique()

In [ ]:
def clean_year_code(year_code):
    if year_code == "Less than 1 year":
        return 0.5
    if year_code == "More than 50 years":
        return 50
    return float(year_code)


df_clean_other["YearsCode"] = df_clean_other["YearsCode"].apply(
    clean_year_code)
df_clean_other["YearsCode"].unique()

In [ ]:
df_clean_other["EdLevel"].unique()

In [ ]:
def clean_education(edu):
    if "Bachelor’s degree" in edu:
        return "Bachelor’s degree"
    if "Master’s degree" in edu:
        return "Master’s degree"
    if "Professional degree" in edu:
        return "More than a Master's degree"
    return "Less than a Bachelor's degree"


df_clean_other["EdLevel"] = df_clean_other["EdLevel"].apply(clean_education)
df_clean_other["EdLevel"].unique()

In [ ]:
df_clean = df_clean_other.reset_index(drop=True)

In [ ]:
from sklearn.preprocessing import LabelEncoder
le_education = LabelEncoder()
df_clean["EdLevel"] = le_education.fit_transform(df_clean["EdLevel"])

le_country = LabelEncoder()
df_clean["Country"] = le_country.fit_transform(df_clean["Country"])
df_clean.head()

In [ ]:
df_clean.value_counts()

In [ ]:
# Split feature
x = df_clean.drop("Salary", axis=1)
y = df_clean["Salary"]

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.linear_model import LinearRegression
lm = LinearRegression()
lm.fit(x, y.values)

lm_y_pred = lm.predict(x)

lm_error = np.sqrt(mean_squared_error(y, lm_y_pred))
lm_error

In [ ]:
from sklearn.tree import DecisionTreeRegressor
dt = DecisionTreeRegressor(random_state=0)
dt.fit(x, y.values)

dt_y_pred = dt.predict(x)

dt_error = np.sqrt(mean_squared_error(y, dt_y_pred))
print("${:,.02f}".format(dt_error))

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(random_state=0)
rf.fit(x, y.values)

rf_y_pred = rf.predict(x)

rf_error = np.sqrt(mean_squared_error(y, rf_y_pred))
print("${:,.02f}".format(rf_error))

In [ ]:
from sklearn.model_selection import GridSearchCV

max_depth = [None, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20]
parameters = {"max_depth": max_depth}

dt_regressor = DecisionTreeRegressor(random_state=0)
dt_gs = GridSearchCV(dt_regressor, parameters,
                     scoring="neg_mean_squared_error")
dt_gs.fit(x, y.values)

dt_regressor = dt_gs.best_estimator_
dt_regressor.fit(x, y.values)

dt_regressor_y_pred = dt_regressor.predict(x)
dt_regressor_error = np.sqrt(mean_squared_error(y, dt_regressor_y_pred))
print("${:,.02f}".format(dt_regressor_error))

In [ ]:
x = np.array([["United States of America", "Master’s degree", 15]])
x

In [ ]:
x[:, 0] = le_country.transform(x[:, 0])
x[:, 1] = le_education.transform(x[:, 1])
x = x.astype(float)
x

In [ ]:
dt_y_pred = dt_regressor.predict(x)
dt_y_pred

In [ ]:
import pickle
data = {"model": dt_regressor, "le_country": le_country,
        "le_education": le_education}
with open("./model/saved_steps.pkl", "wb") as file:
    pickle.dump(data, file)

In [ ]:
with open("./model/saved_steps.pkl", "rb") as file:
    data = pickle.load(file)

dt_regressor_loaded = data["model"]
le_country = data["le_country"]
le_education = data["le_education"]

In [ ]:
y_pred = dt_regressor_loaded.predict(x)
y_pred